In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
ls

Problem Statement:

Pneumonia is an infection in one or both lungs. Bacteria, viruses, and fungi cause it. The infection causes inflammation in the air sacs in your lungs, which are called alveoli.

Now to detection Pneumonia we need to detect Inflammation of the lungs. In this project, you’re challenged to build an algorithm to detect a visual signal for pneumonia in medical images. Specifically, your algorithm needs to automatically locate lung opacities on chest radiographs.

Business Domain Value Automating Pneumonia screening in chest radiographs, providing affected area details through bounding box. Assist physicians to make better clinical decisions or even replace human judgement in certain functional areas of healthcare (eg, radiology).

Project objective In this capstone project, the goal is to build a pneumonia detection system, to locate the position of inflammation in an image.

In [ ]:
#Importing all the standard libraries
#..... array/martrix operations and dataframe libraries
import numpy as np
import pandas as pd
#...........
#.......... Visulaization libraries
import pydicom
import pylab
import matplotlib.pyplot as plt
import seaborn as sn
from skimage.transform import resize

#......
from sklearn.model_selection import train_test_split

# NN model building linraries
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
#...................................................

In [ ]:
# setting path for each of the files
class_path='/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_detailed_class_info.csv'
labels_path='/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv'
Image_train_path='/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/'

In [ ]:
# Files descrition
#1. stage_2_detailed_class_info.csv- contains the information of target label
#2. stage_2_train_labels.csv- contains information on Target and bounding box
#3. stage_2_train_images- contains training images in dcm format

In [ ]:
# Reading class file (first file) as dataframe and check few entries and shape
df_class=pd.read_csv(class_path)
print(df_class.head(10))
print(df_class.shape[0])


In [ ]:
df_class['class'].value_counts()

In [ ]:
#Observation:
# This file ocntains patient Id and repective class ifnormation. 
#. There are 30277 records
# There are three classes- 
#    1. Lung Opacity- Patient havinig pneumonia, 
#    2. Normal- Patient not having pnemonia and not having any other lung problem
#    3. No Lung Opacity/Not Normal- Patient not having pnemonia but having any other lung problem

In [ ]:
df_class.info()

#Observation- There are no null values 

In [ ]:
# checking the number of unique entries with respect to patient ID
print(df_class['patientId'].value_counts().shape[0],'patient cases')

In [ ]:
# # Reading label file (second file) as dataframe and check few entries and shape
df_label=pd.read_csv(labels_path)
print(df_label.head())
print(df_label.shape)

# Observation
#1. There are 30277 lables record (same as the class dataframe)
#2. There are 6 columns - pateint ID (same as order as in class dataframe), bounding box co-ordinates, height and widht and Target label

In [ ]:
# Now lets drop the duplicate cases
df=pd.concat([df_label,df_class.drop('patientId',1)],1)
print(df.shape)
print(df.head())

In [ ]:
# Classes and Targets based on Patient count
df.groupby(['class','Target']).size().reset_index(name='patient_numbers')

In [ ]:
print('Number of duplicate entries accross rows:\n', df[df.duplicated()].count())
print('Number of duplicate Patient Id entries :\n', df[df.duplicated(subset='patientId')].count())
print('Number of unique Patient Id entries: \n', df['patientId'].nunique())
print('Count of various classes: \n',df.groupby('class')['patientId'].nunique())

In [ ]:
# Observation
#1. All the Normal and No Lung Opacity / Not Normal	patients are grouped under Target label 0 (no pnemonia)
#2. Data Imabalance- there are ~30% pneumonia records and rest ~70% no pneumonia
#3  There are no duplicates accross rows
#4. Checking for duplicate patientId's, there are 26684 unique Patient Ids

#--------------------------------------- Exploring training images data -------------------

In [ ]:
# chekcing the type of image file format and total number of images
image_path='/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/'
print(os.listdir(image_path)[0])
import glob
print(len(list(glob.iglob("/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/*.dcm", recursive=True))))

#Observations:
# All the images are in dcm format 
# these image file saved in the Digital Imaging and Communications in Medicine (DICOM) image format. 
#It stores a medical image, such as a CT scan or ultrasound
# There are in total 26684 images which matches with the unique patient IDs. Seems there is no missing image file

In [ ]:
# Checking sample image file for first entry in dataframe which is normal case
print(df.iloc[3])
patientId = df['patientId'][3]
image_path_1='/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/%s.dcm' %patientId
dcm_data=pydicom.read_file(image_path_1)
print(dcm_data)

#Observations:
# dcm file contains metadata information about Patient (sample with no pnemonia): 
#             name, ID, Age, Sex, body part examines, view position, pixel data of image

In [ ]:
#size of image
dcm_data.pixel_array.shape

#observations:
#1. All the 26684 images have same size of 1024 X 1024

In [ ]:
#Plotting the image 
plt.figure(figsize=(12,10))
plt.subplot(121)
plt.title('Pateint- Normal case class')
plt.imshow(dcm_data.pixel_array)
plt.subplot(122)
plt.title('Pateint- Normal case class')
plt.imshow(dcm_data.pixel_array,cmap=plt.cm.gist_gray)

In [ ]:
#... Sample No Lung Opacity / Not Normal case ----------------
print(df.iloc[0])
patientId = df['patientId'][0]
image_path_1='/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/%s.dcm' %patientId
dcm_data=pydicom.read_file(image_path_1)
print(dcm_data)

#Plotting the image 
plt.figure(figsize=(12,10))
plt.subplot(121)
plt.title('Pateint- No Lung Opacity / Not Normal case')
plt.imshow(dcm_data.pixel_array)
plt.subplot(122)
plt.title('Pateint- No Lung Opacity / Not Normal case')
plt.imshow(dcm_data.pixel_array,cmap=plt.cm.gist_gray)

In [ ]:
# Lets us plot one Patient with pnemonia (Target = 1)
print(df.iloc[4])
patientId = df['patientId'][4]
image_path_1='/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/%s.dcm' %patientId
dcm_data=pydicom.read_file(image_path_1)
print(dcm_data)
#Plotting the image 
plt.figure(figsize=(12,10))
plt.subplot(121)
plt.title('Pateint- With pneumonia class')
plt.imshow(dcm_data.pixel_array)
plt.subplot(122)
plt.title('Pateint- With pneumonia class')
plt.imshow(dcm_data.pixel_array,cmap=plt.cm.gist_gray)

In [ ]:
# Function to show to a sample image with overlayed bounding box 
def showImage(row):
    """
    Method to draw single patient with bounding box(es) if present 

    """
    # --- Open DICOM file
    imagePath = "/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/{0}.dcm".format(row['patientId'])
    d = pydicom.read_file(imagePath)
    image = d.pixel_array
    image = np.stack([image] * 3, axis=2)

    if row['Target'] == 1:        
        image = drawbox(image=image, row=row)

    plt.imshow(image, cmap=plt.cm.gist_gray)
    
    
def drawbox(image, row):
    color = np.floor(np.random.rand(3) * 256).astype('int')
    stroke=6
  
    # --- Extract coordinates
    x1 = int(row['x'])
    y1 = int(row['y'])
    y2 = y1 + int(row['height'])
    x2 = x1 + int(row['width'])
    
    #print(x1)
    #print(x2)
    #print(y1)
    #print(y2)
    
    image[y1:y1 + stroke, x1:x2] = color
    image[y2:y2 + stroke, x1:x2] = color
    image[y1:y2, x1:x1 + stroke] = color
    image[y1:y2, x2:x2 + stroke] = color

    return image

In [ ]:
#patient = labels_w_class.iloc[[10]]
patient = list(df.T.to_dict().values())[4]
print("Path : stage_2_train_images/{0}.dcm".format(patient['patientId']))
print("Target : {0}".format(patient['Target']))

plt.figure(figsize=(7,7))
plt.title("Sample Patient - Lung Opacity")
showImage(patient)

#----------------------------------------- Data generation for training, Model Building and training model-----

In [ ]:
# Function to collect three major information (Patient ID, box and corresponding image file path) into a dictonary called parsed
extract_boxes=lambda row: [ row['y'], row['x'], row ['height'], row['width']]
parsed={}
for n, row in df.iterrows():
    pid=row['patientId']
    if pid not in parsed:
        parsed[pid]={
            'dicom': '/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/%s.dcm' % pid,
            'label': row['Target'],
            'boxes':[]
            }
                 
    if parsed[pid]['label']==1:
        parsed[pid]['boxes'].append(extract_boxes(row))

In [ ]:
len(parsed)

In [ ]:
parsed[df['patientId'][7]]

In [ ]:
#............ function to get the images with bounbding box for any given Patient ID
def draw(data):
#    """
#    Method to draw single patient with bounding box(es) if present 

#   """
    # --- Open DICOM file
    d = pydicom.read_file(data['dicom'])
    im = d.pixel_array

    # --- Convert from single-channel grayscale to 3-channel RGB
    im = np.stack([im] * 3, axis=2)

    # --- Add boxes with random color if present
    for box in data['boxes']:
        rgb = np.floor(np.random.rand(3) * 256).astype('int')
        im = overlay_box(im=im, box=box, rgb=rgb, stroke=6)

    pylab.imshow(im, cmap=pylab.cm.gist_gray)
    pylab.axis('off')

def overlay_box(im, box, rgb, stroke=1):
    """
    Method to overlay single box on image

    """
    # --- Convert coordinates to integers
    box = [int(b) for b in box]
    
    # --- Extract coordinates
    y1, x1, height, width = box
    y2 = y1 + height
    x2 = x1 + width

    im[y1:y1 + stroke, x1:x2] = rgb
    im[y2:y2 + stroke, x1:x2] = rgb
    im[y1:y2, x1:x1 + stroke] = rgb
    im[y1:y2, x2:x2 + stroke] = rgb

    return im

In [ ]:
# overalaping bounding box with image for sample pnemonia case
#Plotting the image 
print(df.iloc[4])
patientId = df['patientId'][4]
draw(parsed[patientId])

In [ ]:
import csv
# empty dictionary
pneumonia_locations = {}
# load table
with open(os.path.join(labels_path), mode='r') as infile:
    # open reader
    reader = csv.reader(infile)
    # skip header
    next(reader, None)
    # loop through rows
    for rows in reader:
        # retrieve information
        filename = rows[0]
        location = rows[1:5]
        pneumonia = rows[5]
        # if row contains pneumonia add label to dictionary
        # which contains a list of pneumonia locations per filename
        if pneumonia == '1':
            # convert string to float to int
            location = [int(float(i)) for i in location]
            # save pneumonia location in dictionary
            if filename in pneumonia_locations:
                pneumonia_locations[filename].append(location)
            else:
                pneumonia_locations[filename] = [location]

In [ ]:
len(pneumonia_locations)

In [ ]:
# load and shuffle filenames
folder = Image_train_path
filenames = os.listdir(folder)

In [ ]:
from skimage.transform import resize
import keras
import random
class generator(keras.utils.Sequence):
    
    def __init__(self, folder, filenames, pneumonia_locations=None, batch_size=32, image_size=256, shuffle=True, augment=False, predict=False):
        self.folder = folder
        self.filenames = filenames
        self.pneumonia_locations = pneumonia_locations
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.augment = augment
        self.predict = predict
        self.on_epoch_end()
        
    def __load__(self, filename):
        # load dicom file as numpy array
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array
        # create empty mask
        msk = np.zeros(img.shape)
        # get filename without extension
        filename = filename.split('.')[0]
        # if image contains pneumonia
        if filename in self.pneumonia_locations:
            # loop through pneumonia
            for location in self.pneumonia_locations[filename]:
                # add 1's at the location of the pneumonia
                x, y, w, h = location
                msk[y:y+h, x:x+w] = 1
        # resize both image and mask
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        msk = resize(msk, (self.image_size, self.image_size), mode='reflect') > 0.5
        # if augment then horizontal flip half the time
        if self.augment and random.random() > 0.5:
            img = np.fliplr(img)
            msk = np.fliplr(msk)
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        msk = np.expand_dims(msk, -1)
        return img, msk
    
    def __loadpredict__(self, filename):
        # load dicom file as numpy array
        img = pydicom.dcmread(os.path.join(self.folder, filename)).pixel_array
        # resize image
        img = resize(img, (self.image_size, self.image_size), mode='reflect')
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        return img
        
    def __getitem__(self, index):
        # select batch
        filenames = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        # predict mode: return images and filenames
        if self.predict:
            # load files
            imgs = [self.__loadpredict__(filename) for filename in filenames]
            # create numpy batch
            imgs = np.array(imgs)
            return imgs, filenames
        # train mode: return images and masks
        else:
            # load files
            items = [self.__load__(filename) for filename in filenames]
            # unzip images and masks
            imgs, msks = zip(*items)
            # create numpy batch
            imgs = np.array(imgs)
            msks = np.array(msks)
            return imgs, msks
        
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.filenames)
        
    def __len__(self):
        if self.predict:
            # return everything
            return int(np.ceil(len(self.filenames) / self.batch_size))
        else:
            # return full batches only
            return int(len(self.filenames) / self.batch_size)

In [ ]:
def create_downsample(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 1, padding='same', use_bias=False)(x)
    x = keras.layers.MaxPool2D(2)(x)
    return x

def create_resblock(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    return keras.layers.add([x, inputs])

def create_network(input_size, channels, n_blocks=2, depth=4):
    # input
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(inputs)
    #residual blocks
    for d in range(depth):
        channels = channels * 2
        x = create_downsample(channels, x)
        for b in range(n_blocks):
            x = create_resblock(channels, x)
    # output
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(1, 1, activation='sigmoid')(x)
    outputs = keras.layers.UpSampling2D(2**depth)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
import tensorflow as tf
def iou_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true * y_pred)
    score = (intersection + 1.) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection + 1.)
    return 1 - score

# combine bce loss and iou loss
def iou_bce_loss(y_true, y_pred):
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) + 0.5 * iou_loss(y_true, y_pred)

# mean iou as a metric
def mean_iou(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersect = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3])
    smooth = tf.ones(tf.shape(intersect))
    return tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))

# create network and compiler
model = create_network(input_size=256, channels=32, n_blocks=2, depth=4)
model.compile(optimizer='adam',
              loss=iou_bce_loss,
              metrics=['accuracy', mean_iou])

# cosine learning rate annealing
def cosine_annealing(x):
    lr = 0.001
    epochs = 1
    return lr*(np.cos(np.pi*x/epochs)+1.)/2
learning_rate = tf.keras.callbacks.LearningRateScheduler(cosine_annealing)

model.summary()

In [ ]:
model.layers[-1].output

from keras import backend as K
sample= 

inp = model.input                                           # input placeholder
outputs = model.layers[-1].output         # all layer outputs
functors = [K.function([inp], [out]) for out in outputs]    # evaluation functions

# Testing
layer_outs = [func([sample]) for func in functors]
print(layer_outs)

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("model-{loss:.2f}.h5", monitor="loss", verbose=1, save_best_only=True,
                             save_weights_only=True, mode="max", period=1) # Checkpoint best validation model
#stop = EarlyStopping(monitor="loss", patience=PATIENCE, mode="max") # Stop early, if the validation error deteriorates
#reduce_lr = ReduceLROnPlateau(monitor="loss", factor=0.2, patience=10, min_lr=1e-7, verbose=1, mode="max")

# create train and validation generators
#folder = '../input/stage_1_train_images'
train_gen = generator(folder, filenames, pneumonia_locations, batch_size=64, image_size=256, shuffle=True, augment=True, predict=False)
#valid_gen = generator(folder, valid_filenames, pneumonia_locations, batch_size=32, image_size=256, shuffle=False, predict=False)

history = model.fit_generator(train_gen, callbacks=[checkpoint], epochs=3, workers=4, use_multiprocessing=True)

In [ ]:
model.summary()

In [ ]:
import csv
from skimage import measure


# load and shuffle filenames
folder = '/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_test_images'
test_filenames = os.listdir(folder)
print('n test samples:', len(test_filenames))

# create test generator with predict flag set to True
test_gen = generator(folder, test_filenames, None, batch_size=16, image_size=256, shuffle=False, predict=True)

# create submission dictionary
submission_dict = {}
# loop through testset
for imgs, filenames in test_gen:
    # predict batch of images
    preds = model.predict(imgs)
    # loop through batch
    for pred, filename in zip(preds, filenames):
        # resize predicted mask
        pred = resize(pred, (1024, 1024), mode='reflect')
        # threshold predicted mask
        comp = pred[:, :, 0] > 0.5
        # apply connected components
        comp = measure.label(comp)
        # apply bounding boxes
        predictionString = ''
        for region in measure.regionprops(comp):
            # retrieve x, y, height and width
            y, x, y2, x2 = region.bbox
            height = y2 - y
            width = x2 - x
            # proxy for confidence score
            conf = np.mean(pred[y:y+height, x:x+width])
            # add to predictionString
            predictionString += str(conf) + ' ' + str(x) + ' ' + str(y) + ' ' + str(width) + ' ' + str(height) + ' '
        # add filename and predictionString to dictionary
        filename = filename.split('.')[0]
        submission_dict[filename] = predictionString
    # stop if we've got them all
    if len(submission_dict) >= len(test_filenames):
        break
        
print("Done predicting...")

In [ ]:
# save dictionary as csv file
sub = pd.DataFrame.from_dict(submission_dict,orient='index')
sub.index.names = ['patientId']
sub.columns = ['PredictionString']
sub.to_csv('submission.csv')

In [ ]:
ls

In [ ]:
#----------------------------------------------------------------- RFCNN

from __future__ import division
import random
import pprint
import sys
import time
import numpy as np
#from optparse import OptionParser
import pickle

from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Input
from keras.models import Model

import sys
sys.path.append('/kaggle/working/dt-keras-frcnn')

from keras_frcnn import config, data_generators
from keras_frcnn import losses as losses
import keras_frcnn.roi_helpers as roi_helpers
from keras.utils import generic_utils

train_path = 'annotate.txt'
parser = 'simple'

num_rois = 32
network = 'resnet50'
horizontal_flips = False
vertical_flips =  False
rot_90 = False
num_epochs = 2000
config_filename = "config.pickle"
output_weight_path = './model_frcnn.hdf5'
#input_weight_path = 

train_path = 'annotate.txt'
parser = 'simple'

num_rois = 32
network = 'resnet50'
horizontal_flips = False
vertical_flips =  False
rot_90 = False
num_epochs = 2000
config_filename = "config.pickle"
output_weight_path = './model_frcnn.hdf5'
#input_weight_path = 